In [155]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import statsmodels.formula.api as smf
import statsmodels.api as sm
%matplotlib inline

В этом уроке применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные. 

In [3]:
cars = pd.read_csv("https://stepik.org/media/attachments/lesson/387691/cars.csv")

1.	Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.

In [5]:
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [9]:
cars.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [13]:
cars.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

2.	Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.

In [40]:
cars['company'] = cars.CarName.str.split(' ').str[0]

In [101]:
cars.drop("car_ID",axis="columns",inplace = True)

In [105]:
cars.drop("CarName",axis="columns",inplace = True)

In [107]:
cars.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [109]:
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [117]:
cars.loc[cars.company == 'vokswagen',"company"] = "volkswagen"
cars.loc[cars.company == 'vw',"company"] = "volkswagen"
cars.loc[cars.company == 'Nissan',"company"] = "nissan"
cars.loc[cars.company == 'porcshce',"company"] = "porsche"
cars.loc[cars.company == 'toyouta',"company"] = "toyota"

In [119]:
# Второй способ замены через replace
cars.company.replace("maxda","mazda")

0      alfa-romero
1      alfa-romero
2      alfa-romero
3             audi
4             audi
          ...     
200          volvo
201          volvo
202          volvo
203          volvo
204          volvo
Name: company, Length: 205, dtype: object

In [111]:
cars.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

3.	Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными.

In [113]:
cars.dtypes

symboling             int64
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
company              object
dtype: object

In [121]:
cars_predictors = cars[['company','fueltype','aspiration','carbody','drivewheel',
                       'wheelbase','carlength','carwidth','curbweight','enginetype',
                       'cylindernumber','enginesize','boreratio','horsepower','price']]

In [125]:
cars_predictors.select_dtypes(include=['float64','int64']).corr()

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294,0.577816
carlength,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623,0.682920
carwidth,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732,0.759325
curbweight,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739,0.835305
enginesize,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769,0.874145
boreratio,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677,0.553173
horsepower,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000,0.808139
price,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139,1.000000


4.	Преобразуйте категориальные переменные с помощью One Hot Encoding

In [133]:
cars_lr = pd.get_dummies(cars, drop_first = True)
cars_lr.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,company_nissan,company_peugeot,company_plymouth,company_porsche,company_renault,company_saab,company_subaru,company_toyota,company_volkswagen,company_volvo
0,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,False,False,False,False,False,False,False,False,False,False
1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,False,False,False,False,False,False,False,False,False,False
2,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,...,False,False,False,False,False,False,False,False,False,False
3,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,...,False,False,False,False,False,False,False,False,False,False
4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,...,False,False,False,False,False,False,False,False,False,False


5.	Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель? (\(R^2\))

In [136]:
model = smf.ols(formula='price ~ horsepower', data=cars_lr).fit()

In [138]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Tue, 10 Dec 2024   Prob (F-statistic):           1.48e-48
Time:                        18:55:29   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.000   -5555.163   -1888.360
horsepower   163.2631      8.351     19.549      0.000     146.796     179.730
==============================================================================
Omnibus:                       47.741   Durbin-Watson:                   0.792
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.702
Skew:                           1.141   Prob(JB):                     1.22e-20
Kurtosis:                       5.352   Cond. No.                         314.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

6.	Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в \(R^2\), коэффициентах и их значимости. Какую модель лучше оставить? 

In [141]:
cars_lr.dtypes

symboling               int64
wheelbase             float64
carlength             float64
carwidth              float64
carheight             float64
                       ...   
company_saab             bool
company_subaru           bool
company_toyota           bool
company_volkswagen       bool
company_volvo            bool
Length: 65, dtype: object

In [151]:
X = cars_lr.drop(['price'],axis = 'columns') # предикторы
y = cars_lr['price'] # целевая переменная

In [157]:
X = sm.add_constant(X)

In [161]:
 np.asarray(cars_lr)

array([[3, 88.6, 168.8, ..., False, False, False],
       [3, 88.6, 168.8, ..., False, False, False],
       [1, 94.5, 171.2, ..., False, False, False],
       ...,
       [-1, 109.1, 188.8, ..., False, False, True],
       [-1, 109.1, 188.8, ..., False, False, True],
       [-1, 109.1, 188.8, ..., False, False, True]], dtype=object)

In [163]:
model_1 = sm.OLS(y, X.astype('float')).fit()
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.952
Method:                 Least Squares   F-statistic:                     68.86
Date:                Tue, 10 Dec 2024   Prob (F-statistic):           6.09e-82
Time:                        19:08:57   Log-Likelihood:                -1784.9
No. Observations:                 205   AIC:                             3692.
Df Residuals:                     144   BIC:                             3895.
Df Model:                          60                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.822e+04   1.03e+04     -1.774      0.078   -3.85e+04    2077.518
symboling              -231.3220    251.239     -0.921      0.359    -727.914     265.270
wheelbase               236.9919     90.212      2.627      0.010      58.681     415.303
carlength              -140.8553     49.363     -2.853      0.005    -238.425     -43.286
carwidth                716.3306    223.025      3.212      0.002     275.505    1157.157
carheight              -276.1946    137.962     -2.002      0.047    -548.886      -3.503
curbweight                6.0038      1.676      3.581      0.000       2.690       9.317
enginesize               98.6090     25.392      3.884      0.000      48.420     148.798
boreratio             -3412.7584   1833.344     -1.861      0.065   -7036.500     210.983
stroke                -1214.0638    988.547     -1.228      0.221   -3168.001     739.874
compressionratio       -801.7986    487.495     -1.645      0.102   -1765.370     161.773
horsepower               -3.9024     23.875     -0.163      0.870     -51.093      43.289
peakrpm                   2.3955      0.644      3.721      0.000       1.123       3.668
citympg                  -7.2882    133.601     -0.055      0.957    -271.360     256.784
highwaympg              107.6208    114.212      0.942      0.348    -118.128     333.369
fueltype_gas          -1.511e+04   5278.101     -2.862      0.005   -2.55e+04   -4672.600
aspiration_turbo       1936.0410    816.347      2.372      0.019     322.470    3549.612
doornumber_two         -192.0887    482.171     -0.398      0.691   -1145.136     760.959
carbody_hardtop       -2265.6906   1186.706     -1.909      0.058   -4611.304      79.923
carbody_hatchback     -3087.0465   1102.740     -2.799      0.006   -5266.695    -907.397
carbody_sedan         -2442.6160   1187.699     -2.057      0.042   -4790.192     -95.040
carbody_wagon         -2648.1742   1282.503     -2.065      0.041   -5183.137    -113.211
drivewheel_fwd          -49.4006    878.945     -0.056      0.955   -1786.701    1687.900
drivewheel_rwd          518.3113   1128.052      0.459      0.647   -1711.368    2747.991
enginelocation_rear    7836.6903   1919.351      4.083      0.000    4042.949    1.16e+04
enginetype_dohcv      -3665.2231   4654.124     -0.788      0.432   -1.29e+04    5534.003
enginetype_l           -444.9455   1629.748     -0.273      0.785   -3666.264    2776.373
enginetype_ohc          847.6017   1199.510      0.707      0.481   -1523.320    3218.524
enginetype_ohcf        2765.8324   1059.360      2.611      0.010     671.928    4859.737
enginetype_ohcv       -2565.4733   1241.637     -2.066      0.041   -5019.663    -111.284
enginetype_rotor       2681.3535   2249.210      1.192      0.235   -1764.379    7127.086
cylindernumber_five   -

In [167]:
# Удаляем марки автомобилей
cols_to_keep = cars_lr.columns[~cars_lr.columns.str.startswith('company_')]
cols_to_keep

Index(['symboling', 'wheelbase', 'carlength', 'carwidth', 'carheight',
       'curbweight', 'enginesize', 'boreratio', 'stroke', 'compressionratio',
       'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price',
       'fueltype_gas', 'aspiration_turbo', 'doornumber_two', 'carbody_hardtop',
       'carbody_hatchback', 'carbody_sedan', 'carbody_wagon', 'drivewheel_fwd',
       'drivewheel_rwd', 'enginelocation_rear', 'enginetype_dohcv',
       'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf', 'enginetype_ohcv',
       'enginetype_rotor', 'cylindernumber_five', 'cylindernumber_four',
       'cylindernumber_six', 'cylindernumber_three', 'cylindernumber_twelve',
       'cylindernumber_two', 'fuelsystem_2bbl', 'fuelsystem_4bbl',
       'fuelsystem_idi', 'fuelsystem_mfi', 'fuelsystem_mpfi',
       'fuelsystem_spdi', 'fuelsystem_spfi'],
      dtype='object')

In [171]:
X_2 = cars_lr[cols_to_keep].drop('price',axis = 'columns')
y_2 = cars_lr['price']

In [173]:
X_2 = sm.add_constant(X_2)

In [177]:
model_2 = sm.OLS(y_2,X_2.astype('float')).fit()
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     61.79
Date:                Tue, 10 Dec 2024   Prob (F-statistic):           4.86e-80
Time:                        19:12:50   Log-Likelihood:                -1844.9
No. Observations:                 205   AIC:                             3774.
Df Residuals:                     163   BIC:                             3913.
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.876e+04   1.02e+04     -1.844      0.067   -3.89e+04    1333.403
symboling                73.8778    238.601      0.310      0.757    -397.269     545.025
wheelbase                48.8156     96.753      0.505      0.615    -142.235     239.866
carlength               -61.3023     48.754     -1.257      0.210    -157.572      34.968
carwidth                693.5728    239.396      2.897      0.004     220.855    1166.290
carheight                89.4347    127.847      0.700      0.485    -163.015     341.885
curbweight                3.9419      1.715      2.299      0.023       0.556       7.328
enginesize              117.3636     25.995      4.515      0.000      66.033     168.694
boreratio             -1882.3376   1597.670     -1.178      0.240   -5037.136    1272.461
stroke                -4453.7640    900.932     -4.944      0.000   -6232.766   -2674.762
compressionratio       -800.3216    525.882     -1.522      0.130   -1838.742     238.099
horsepower                9.7913     22.272      0.440      0.661     -34.187      53.770
peakrpm                   2.2021      0.619      3.555      0.000       0.979       3.425
citympg                -147.7491    147.375     -1.003      0.318    -438.759     143.261
highwaympg              191.6090    134.739      1.422      0.157     -74.451     457.669
fueltype_gas          -1.527e+04   5389.519     -2.833      0.005   -2.59e+04   -4626.644
aspiration_turbo       1626.0175    885.607      1.836      0.068    -122.725    3374.760
doornumber_two          187.6006    585.406      0.320      0.749    -968.356    1343.557
carbody_hardtop       -3207.0350   1375.969     -2.331      0.021   -5924.058    -490.012
carbody_hatchback     -3280.8341   1222.941     -2.683      0.008   -5695.684    -865.985
carbody_sedan         -2151.5954   1332.017     -1.615      0.108   -4781.830     478.639
carbody_wagon         -3265.6309   1455.394     -2.244      0.026   -6139.487    -391.775
drivewheel_fwd           74.0486   1040.473      0.071      0.943   -1980.495    2128.592
drivewheel_rwd         1032.7674   1205.057      0.857      0.393   -1346.768    3412.302
enginelocation_rear    7695.0237   2535.529      3.035      0.003    2688.305    1.27e+04
enginetype_dohcv      -7189.2910   4673.503     -1.538      0.126   -1.64e+04    2039.123
enginetype_l          -1051.0793   1607.967     -0.654      0.514   -4226.211    2124.053
enginetype_ohc         3125.6464    908.839      3.439      0.001    1331.031    4920.262
enginetype_ohcf        1233.6574   1571.770      0.785      0.434   -1869.999    4337.314
enginetype_ohcv       -5605.2579   1246.936     -4.495      0.000   -8067.488   -3143.027
enginetype_rotor        -34.6261   2252.494     -0.015      0.988   -4482.456    4413.204
cylindernumber_five   -